In [1]:
"""Script permettant d'explorer les données de travaux sur Rennes, au format JSON"""
################################################################################
# fichier  : td1.py
# Auteur : RAKOTOSON Loic
################################################################################

"Script permettant d'explorer les données de travaux sur Rennes, au format JSON"

In [2]:
################################################################################
# Importation de fonctions externes :
import json
from datetime import datetime, timedelta
# Definition locale de fonctions :

In [3]:
def initListeChantiers(data):
    listeChantier = [Chantier(dico["properties"]) for dico in data["features"]]
    return listeChantier

In [4]:
def listeChantiersEnCours(listeChantiers,
                          date=datetime.now().strftime("%Y-%m-%d %H:%M:%S")):
    liste = [chantier for chantier in listeChantiers if chantier.enCours(date)]
    return liste

In [5]:
def affichePlanningChantiers(
        listeChantiers, ID, date=datetime.now().strftime("%Y-%m-%d %H:%M:%S")):

    dateformat = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    listequartier = [
        chantier for chantier in listeChantiers if chantier.quartierID == ID
    ]

    chaine = "Planning des chantiers pour le quartier {}".format(
        Chantier.quartiers[ID])
    for chantier in listequartier:
        chaine += "\n {} ({}): ".format(chantier.localisation, chantier.type)
        if chantier.enCours(date):
            fin = chantier.fin - dateformat
            chaine += "chantier en cours (fin dans {} jours et {} heures)".format(
                fin.days, fin.seconds // 3600)
        elif chantier.aVenir(date):
            debut = dateformat - chantier.debut
            chaine += "chantier à venir (début dans {} jours et {} heures)".format(
                debut.days, fin.seconds // 3600)
        elif chantier.termine(date):
            fin = dateformat - chantier.fin
            chaine += "chantier terminé depuis {} jours et {} heures)".format(
                fin.days, fin.seconds // 3600)
        else:
            chaine += "Manque d'informations"
    print(chaine)

In [6]:
def dumpListeChantiersJSON(output, listeChantier):
    data = {
        "chantiers":
        [chantier.jsonDictionnary() for chantier in listeChantier]
    }
    with open(output + ".json", 'w', encoding="UTF-8") as outfile:
        json.dump(data, outfile, ensure_ascii=False)

In [7]:
################################################################################
# Definition des classes :

In [8]:
class Chantier():
    quartiers = dict()

    def __init__(self, properties):
        quartier = properties["quartier"]

        self.id = properties["id"]
        self.quartierID = int(quartier.split(" - ")[0])
        self.localisation = properties["localisation"]
        self.type = properties["type"]
        self.libelle = properties["libelle"]
        self.perturbation = properties["niv_perturbation"]

        if self.quartierID not in self.__class__.quartiers:
            self.__class__.quartiers[self.quartierID] = "/".join(
                quartier.split(" - ")[1:])

        self.debut = datetime.strptime(properties["date_deb"],
                                       "%Y-%m-%d %H:%M:%S")
        self.fin = datetime.strptime(properties["date_fin"],
                                     "%Y-%m-%d %H:%M:%S")

    def __repr__(self):
        chaine = "{}, {} du {} au {}: {} ({}, {})".format(
            self.quartierID, self.localisation,
            self.debut.strftime("%Y/%m/%d %H:%M:%S"),
            self.fin.strftime("%Y/%m/%d %H:%M:%S"), self.libelle, self.type,
            self.perturbation)
        return chaine

    def enCours(self, date):
        date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return self.debut < date < self.fin

    def termine(self, date):
        date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return date > self.fin

    def aVenir(self, date):
        date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        return date < self.debut

    def jsonDictionnary(self):
        dico = dict()
        dico["id"] = self.id
        dico["quartier"] = self.__class__.quartiers[self.quartierID]
        dico["localisation"] = self.localisation
        dico["libelle"] = self.libelle
        dico["type"] = self.type
        dico["perturbation"] = self.perturbation
        dico["debut"] = self.debut.strftime("%Y/%m/%d %H:%M:%S")
        dico["fin"] = self.fin.strftime("%Y/%m/%d %H:%M:%S")
        return dico

In [9]:
################################################################################
# Corps principal du programme :

# EXO 1
Lecture et récupération des données

In [10]:
with open("travaux.json", "r", encoding="UTF-8") as json_file:
    travaux = json.load(json_file)

# EXO 2
Classe Chantier

In [11]:
listeChantier = initListeChantiers(travaux)
print(listeChantier[0])

9, Rue de Redon section Surcouf/Malakoff du 2014/02/17 00:00:00 au 2020/07/31 00:00:00: Métro ligne b - GC Station Mabilais (Route barrée, Secteur à éviter)


# EXO 3
Gestion des dates

In [12]:
listeChantier[0].enCours(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

True

# EXO 4
Utilisation Json

In [13]:
affichePlanningChantiers(listeChantier, 1)

Planning des chantiers pour le quartier Centre
 Boulevard Magenta (Fermeture sens nord-sud): chantier en cours (fin dans 222 jours et 0 heures)
 Place Sainte-Anne (Route barrée): chantier en cours (fin dans 191 jours et 0 heures)
 Rue Descartes (Route barrée): chantier en cours (fin dans 69 jours et 0 heures)
 Rue Albert Aubry (Route barrée): chantier en cours (fin dans 69 jours et 0 heures)
 Avenue Jean Janvier (Chaussée rétrécie): chantier en cours (fin dans 39 jours et 0 heures)
 Avenue Jean Janvier (Chaussée rétrécie): chantier en cours (fin dans 69 jours et 0 heures)
 Rue des Francs Bourgeois (Route barrée): chantier en cours (fin dans 9 jours et 0 heures)
 Boulevard de Beaumont (Fermeture sens est-ouest): chantier en cours (fin dans 94 jours et 0 heures)
 Place de la Gare (Route barrée): chantier en cours (fin dans 51 jours et 0 heures)
 Rue du Capitaine Maignan (Route barrée): chantier terminé depuis 4 jours et 23 heures)
 Rue de la Cochardière (Route barrée): chantier terminé d

In [14]:
dumpListeChantiersJSON("export", listeChantier)